 - 线性分类器
	 - 评分函数 score function
	 - 线性分类器的理解和解释
 - 损失函数 loss function
	 -  多类SVM
   	 - softmax分类器
   	 -  SVM和softmax比较

----------
KNN分类器存在的不足：

- 分类器必须记住所有的训练数据并存储起来，以便未来测试数据比较，需要很大的空间

- 对一个测试数据需要与所有的训练数据进行比较，需要很大的计算资源和时间

为此，我们需要一种更好的方法：线性分类器

----------


 - 线性分类器
==
由两部分组成：评分函数score function和损失函数loss function。该方法可以转换为一个最优化问题，在优化过程中，将通过更新评分函数的参数来最小化损失函数值。

**评分函数 score function**    $$f(x_i,W,b) = Wx_i+b$$
我们的目标就是通过学习得到这些参数W,b，使得计算出来的分类值情况和训练集中图像数据的真实标签相符。
相比之下：
 - 训练数据得到W,b，一旦训练完成，训练数据就可以丢掉，留下参数即可
 - 只需要一个矩阵乘法和一个矩阵加法就能对一个测试数据进行分类。

我们可以这样理解线性分类器：
![这里写图片描述](http://img.blog.csdn.net/20180131095637041?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcGFueGlhb3hpZQ==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)
图像像素是三维的，可以拉伸为一个列向量。
假设总共有10个分类，图像大小是32×32×3，则对于一张图片
$W.shape = (10,3072)$
$x.shape = (3072,1)$
$b.shape = (10,1)$
得到结果是(10，1),也就是对这张图片给个分类有一个值，比重最大的对应其所属的分类。


> **怎么理解呢？**我是这样理解的
> 
> W有10行，可以将每一行看成一个概率分布，其概率分布越接近$x$的概率分布，则点乘之后的值越大。我们训练的目的就是将W第一行的值接近于cat的像素分布。第二行接近dog的像素分布。。。

如果训练数据中有1000张船、1000张猫、1000张船的图像数据。总共10000张图像数据
$W.shape = (10,3072)$
$x.shape = (3072,10000)$
$b.shape = (10,1)$
得到(10,10000)，其中cat标签对应的(10,1000)中cat的比重应该最大，也就是这样训练得到W对应cat那一行的权重。当然W可能不能满足所有的，这样就是造成不准确嘛～～

> 实际上有10类就意味着有10个线性分类器～每一行对应一个分类器。

另一种解释：关于权重W的另一个解释是它的每一行对应着一个分类的模板（有时候也叫作原型）。一张图像对应不同分类的得分，是通过使用内积（也叫点积）来比较图像和模板，然后找到和哪个模板最相似。从这个角度来看，线性分类器就是在利用学习到的模板，针对图像做模板匹配。从另一个角度来看，可以认为还是在高效地使用k-NN，不同的是我们没有使用所有的训练集的图像来比较，而是每个类别只用了一张图片（这张图片是我们学习到的，而不是训练集中的某一张），而且我们会使用（负）内积来计算向量间的距离，而不是使用L1或者L2距离。


**偏差和权重合并技巧**
$f(x_i,W,b) = Wx_i+b$
通常可以将两个参数放到同一个矩阵，$x_i$增加一个维度，常量1。
![这里写图片描述](http://img.blog.csdn.net/20180131102940293?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcGFueGlhb3hpZQ==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

**图像预处理**
零均值中心化：很重要～梯度下降后在解释

----------

损失函数
==
要使得其与原始训练数据通过score function得到的结果与真实类别一致，我们需要调整参数权重W和偏差b. 怎么调整是梯度下降的事儿～那么调整之前，我们需要先衡量下对结果的不满意程度，然后适当调整对吧～～这就是损失函数干的事儿～

**多类支持向量机损失 Multiclass Support Vector Mashine Loss**


$L_i = \sum_{j\ne y_i}^{10}max(0,s_j - s_{y_i}+\Delta)$

> 怎么理解这个公式？
> 公式可以这么写：$L_i = \sum_{j\ne y_i}^{10}max(0,s_j - (s_{y_i}-\Delta))$
> 对第i张图像数据，其最后结果有10个，其中$s_{y_i}$表示其真实标签对应的值，那么其他非真实分类的结果凡是大于$s_{y_i}-\Delta$这个值的，都对最后的结果$L_i$产生影响，比这个值小的就没事～
> 如下图所示：
![这里写图片描述](http://img.blog.csdn.net/20180131113618230?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcGFueGlhb3hpZQ==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

那么对于评分函数，我们可以将公式稍微修改下～
$L_i = \sum_{y\ne y_i}^{10}max(0,w_j^Tx_i - w_{y_j}^Tx_i+\Delta)$

这里提一下的属于是关于0的阀值：$max(0,-)$函数，它常被称为折叶损失（hinge loss）。有时候会听到人们使用平方折叶损失SVM（即L2-SVM），它使用的是$max(0,-)^2$，将更强烈（平方地而不是线性地）地惩罚过界的边界值。不使用平方是更标准的版本，但是在某些数据集中，平方折叶损失会工作得更好。可以通过交叉验证来决定到底使用哪个。


**正则化（Regularization）：**

cs231n笔记：一个简单的例子：如果W能够正确分类所有数据，即对于每个数据，损失值都是0。那么当$\lambda>1$时，任何数乘$\lambda W$都能使得损失值为0，因为这个变化将所有分值的大小都均等地扩大了，所以它们之间的绝对差值也扩大了。
> 
> 我有个其他的想法：我们知道W（10，3072）是代表10个分类器的，假设它与cat的图像数据（1，3072）点乘，得到10个数据${s_1}...{s_{10}}$，假设cat分类器W0得到$s_1$，dog分类器W1得到$s_2$，显然cat和dog本来就很像，要是$s_2$减小，要抑制其分类器即W1中的某些值。。。总的下来就是cat的图像会影响dog分类器？？？。。

解决此问题的方法是加一个正则化惩罚项（regularization penalty）$R(W)$部分。
L2范式：$R(W) = \sum_k\sum_lW_{k,l}^2$
这里是将所有的参数平方后累加得到的一个值。正则化函数不是数据的函数，仅仅基于权重～
SVM损失函数有两部分组成：
![这里写图片描述](http://img.blog.csdn.net/20180131122124827?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcGFueGlhb3hpZQ==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)
展开是：
$L = \frac{1}{N}\sum_i\sum_{y\ne y_i}[0,f(x_i;W)_j-f(x_i;W)_{y_j} + \Delta] +\lambda\sum_k\sum_lW_{k,l}^2$

>  **$\lambda是超参数$**，通过交叉验证来获取？？？
>  引入L2惩罚后，SVM就有了最大边界（maxmargin）这一性质。其最好的性质就是对大数值权重进行惩罚，可以提高泛化能力，没有那个维度能独立对整体有过很大的影响。鼓励分类器最终将所有维度上的特征都用起来，而不是强烈依赖其中少数几个维度，提升分类器的泛化能力，并避免过拟合。

> **设置Delta：**超参数\Delta它应该被设置成什么值？需要通过交叉验证来求得吗？现在看来，该超参数在绝大多数情况下设为\Delta=1.0都是安全的。超参数\Delta和\lambda看起来是两个不同的超参数，但实际上他们一起控制同一个权衡：**即损失函数中的数据损失和正则化损失之间的权衡。**理解这一点的关键是要知道，权重W的大小对于分类分值有直接影响（当然对他们的差异也有直接影响）：当我们将W中值缩小，分类分值之间的差异也变小，反之亦然。因此，不同分类分值之间的边界的具体值（比如\Delta=1或\Delta=100）从某些角度来看是没意义的，因为权重自己就可以控制差异变大和缩小。也就是说，真正的权衡是我们允许权重能够变大到何种程度（通过正则化强度\lambda来控制）。

**Softmax分类器** 
Softmax分类器就可以理解为逻辑回归分类器面对多个分类的一般化归纳。
函数映射保持不变，将这些评分值视为每个分类的未归一化的对数频率。即交叉熵损失函数(cross-entropy loss). 公式如下：
$L_i = -\log(\frac{e^{f_{y_i}}}{\sum_je^{f_j}})$
整个数据集的损失值：$L = \frac{1}{N}\sum_iL_i +\lambda R(W) $
其中：$f_j(z) = \frac{e^{z_j}}{\sum_ke^{z_k}}$称为softmax函数，骑术如是一个向量，sofamax对其进行压缩，输出一个向量，其中每个值在0到1之间，且所有和为1。
> **从信息论的视角来理解：**
> 信息量/信息熵（熵）/交叉熵/条件熵
> **信息量：**一个事件的信息量就是这个时间发生的概率的负对数，概率越大，所带来的信息就越少嘛。至于为什么是负对数，就要问香农了。。起码要满足$P(X)=1$时信息量为0，且始终大于0 $$-\log P(X)$$
> **信息熵，**也就是熵，是随机变量不确定性的度量，依赖于事件X的概率分布。即信息熵是信息量的期望。即求离散分布列的期望～～$$H(p) = -\sum_{i=1}^np_i\log p_i$$
> **交叉熵：**回归到分类问题来，我们通过score function得到一个结果（10，1），通过softmax函数压缩成0到1的概率分布，我们称为$q_i$吧，$$H(p,q) = -\sum_{i=1}^nq_i\log p_i$$就是我们所说的交叉熵，通过分析可以发现：$H(p,q)>=H(p)$恒成立，当且仅当$q_i$分布和$p_i$相同时，两者相等。
> 我们这里因为$p_i$是真实分布，一旦训练数据确定，其就是定植，因此就省去了，得到了cross-entroy function~~~
> **相对熵：**跟交叉熵是同样的概念，$D(p||q)=H(p,q)-H(p)$，又称为KL散度，表征两个函数或概率分布的差异性，差异越大则相对熵越大～～～

实操需要注意的事项：
$\sum_je^{f_j}$和$e^{f_{y_i}}$是指数函数，数值可能很大，所以使用归一化
![这里写图片描述](http://img.blog.csdn.net/20180131151612768?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcGFueGlhb3hpZQ==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)
其中通常将常数C设置为$logC = -max(f_j)$ 这样$e^{f_j+\log C}$最大值也就是1，最小值是0。这样压缩数据并没有改变对应的比重哦～～


SVM和Softmax的比较
SVM的计算是无标定的，只给出了一个分类最终结果，而且难以针对所有分类的评分值给出直观解释。Softmax分类器则不同，它允许我们计算出对于所有分类标签的“可能性”。
但这个“可能性”并不是绝对的，其集中或离散程度是由正则化参数$\lambda$直接决定的，$\lambda$是超参数，能直接控制的。如果正则化参数$\lambda$越大，那么权重就会被惩罚的越多（因为$L_i$中的regularization loss 不能太大）。随着正则化参数$\lambda$不断变大，权重数值回越来越小，最后接近于均匀分布。这就是说，softmax分类器算出来的概率最好是看成一种对于分类正确性的自信。和SVM一样，数字间相互比较得出的大小顺序是可以解释的，但其绝对值则难以直观解释。
